<center>

<h1> Twitter Reacts: Gun Violence </h1>
<h3>Priyanka Khanal </h3>
<h3>April 23, 2019</h3>

</center>

<img src="mainpic.jpg">
<br>
Gun violence has increased significantly in the past decade. Not just in United States, gun violence is increasing all over the world. This project analyzes the rectioon of twitter to two different gun violence incidents of recent times. The first incident took place in Stoneman Douglas High School shooting in Parkland, Florida on February 14, 2018. This school shooting is deadliest high school shooting spree in the history of United States which led to 17 deaths and 14 non-fatal injuries.

The second incident took place consequitively at two mosques in Christchurch, New Zealand on March 15, 2019. This terriost attack led to 50 deaths and 50 non-fatal injuries.

### Importing Various Python Libraries 

In [34]:
from twitter_keys import consumer_key, consumer_secret, access_token, access_secret
import tweepy #for twitter API
import pickle
import pandas as pd # To Handle Data
import numpy as np #for number computing
from collections import Counter
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from textblob import TextBlob 
from wordcloud import WordCloud, STOPWORDS 
from IPython.display import Image
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\s524702\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [35]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

## Functions being used

In [36]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt  

In [37]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0:
        return 0
    else:
        return -1

In [38]:
def plot_sent(tweets):
    texts=[tweet.full_text for tweet in tweets]
    sentiments=[analyzer.polarity_scores(t) for t in texts]
    pos_x=[s['pos'] for s in sentiments]
    neu_y=[s['neu'] for s in sentiments]
    neg_z=[s['neg'] for s in sentiments]
    return pos_x,neu_y, neg_z

In [39]:
def random_color_func(word=None, font_size=None, position=None,  orientation=None, font_path=None, random_state=None):
    h = int(360.0 * 21.0 / 255.0)
    s = int(100.0 * 255.0 / 255.0)
    l = int(100.0 * float(random_state.randint(60, 120)) / 255.0)

    return "hsl({}, {}%, {}%)".format(h, s, l)

## Pickling Data

In [40]:
parkland=[]
for tweet in tweepy.Cursor(api.search,q = "#marchforourlives",lang="en",tweet_mode='extended',exclude_replies=True).items(10000):
    parkland.append(tweet)
#with open("parkland.pkl", 'wb') as f:
   # pickle.dump(parkland, f)
with open('parkland.pkl', 'rb') as f:
    parkland = pickle.load(f)

In [41]:
christchurch=[]
for tweet in tweepy.Cursor(api.search,q='#christchurchshooting',lang='en',tweet_mode='extended',exclude_replies=True).items(10000):
    christchurch.append(tweet)
#with open("christchurch.pkl", 'wb') as f:
   # pickle.dump(christchurch, f)
with open('christchurch.pkl', 'rb') as f:
    christchurch = pickle.load(f)


## Printing in a table format

In [42]:
data_pl = pd.DataFrame(data=[tweet.full_text for tweet in parkland], columns=['parklandTweets'])
data_pl['parklandlen']  = np.array([len(tweet.full_text) for tweet in parkland])
data_pl['parklandDate'] = np.array([tweet.created_at for tweet in parkland])
data_pl['parklandSource'] = np.array([tweet.source for tweet in parkland])
data_pl['parklandLikes']  = np.array([tweet.favorite_count for tweet in parkland])
data_pl['parklandRTs']    = np.array([tweet.retweet_count for tweet in parkland])
data_pl['users_locs'] = np.array([ tweet.user.location for tweet in parkland])
data_pl['parklandClean'] = np.vectorize(remove_pattern)(data_pl['parklandTweets'], "@[\w]*")
data_pl['parklandClean'] = data_pl['parklandClean'].str.replace("[^a-zA-Z#]", " ")
data_pl['parklandClean']= data_pl['parklandClean'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>4]))
data_pl['parklandSA'] = np.array([analyze_sentiment(tweet) for tweet in data_pl['parklandClean'] ])
data_pl.head()

,parklandTweets,parklandlen,parklandDate,parklandSource,parklandLikes,parklandRTs,users_locs,parklandClean,parklandSA
0,RT @nowthisnews: 735 people reportedly die fro...,140,2019-04-23 07:01:08,Twitter for iPhone,0,1538,,people reportedly violence every #MarchForOurL...,0
1,I BELIEVE in you guys! #MarchForOurLives https...,64,2019-04-23 06:52:08,Twitter for Android,0,0,Tacoma,BELIEVE #MarchForOurLives https MsnLnDH,0
2,RT @cantrell4action: Am I next? @NRA #MarchFor...,78,2019-04-23 06:49:36,Twitter for iPhone,0,254,,#MarchForOurLives https dnkGeM,0
3,RT @cantrell4action: Am I next? @NRA #MarchFor...,78,2019-04-23 06:41:40,Twitter for iPad,0,254,"Munich, Germany",#MarchForOurLives https dnkGeM,0
4,RT @cantrell4action: Am I next? @NRA #MarchFor...,78,2019-04-23 06:25:32,Twitter for iPhone,0,254,,#MarchForOurLives https dnkGeM,0


In [43]:
data = pd.DataFrame(data=[tweet.full_text for tweet in christchurch], columns=['christchurchTweets'])
data['christchurchlen']  = np.array([len(tweet.full_text) for tweet in christchurch])
data['christchurchDate'] = np.array([tweet.created_at for tweet in christchurch])
data['christchurchSource'] = np.array([tweet.source for tweet in christchurch])
data['christchurchLikes']  = np.array([tweet.favorite_count for tweet in christchurch])
data['christchurchRTs']    = np.array([tweet.retweet_count for tweet in christchurch])
data['christhurchusers_locs'] = np.array([ tweet.user.location for tweet in christchurch])
data['christchurchClean'] = np.vectorize(remove_pattern)(data['christchurchTweets'], "@[\w]*")
data['christchurchClean'] = data['christchurchClean'].str.replace("[^a-zA-Z#]", " ")
data['christchurchClean']= data['christchurchClean'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>4]))
data['christchurchSA'] = np.array([analyze_sentiment(tweet) for tweet in data['christchurchClean'] ])
data.head()

,christchurchTweets,christchurchlen,christchurchDate,christchurchSource,christchurchLikes,christchurchRTs,christhurchusers_locs,christchurchClean,christchurchSA
0,"A special visa category, Christchurch Response...",274,2019-04-23 05:22:37,Twitter for Android,0,0,New Zealand,special category Christchurch Response created...,1
1,New Zealand Mosque Shooting and the Far Right ...,270,2019-04-23 05:00:04,Smedian Network,0,0,Currently: 🇨🇦,Zealand Mosque Shooting Right #medium #blog #e...,1
2,Can't wait for her next #outfit @jacindaardern...,204,2019-04-22 22:08:27,Instagram,0,0,Melbourne,#outfit consult #waleedaly #newzealand #christ...,0
3,RT @Timmy_John17: To all the men and women in ...,139,2019-04-22 20:53:39,Twitter Web App,0,2,,women Christchurch police emergency services d...,0
4,New Zealand Mosque Shooting and the Far Right ...,270,2019-04-22 15:55:03,Smedian Network,0,0,Currently: 🇨🇦,Zealand Mosque Shooting Right #medium #blog #e...,1


## Sentiment Analysis

In [46]:
CCpos_tweets = [ tweet for index, tweet in enumerate(data['christchurchClean']) if data['christchurchSA'][index] > 0]
CCneu_tweets = [ tweet for index, tweet in enumerate(data['christchurchClean']) if data['christchurchSA'][index] == 0]
CCneg_tweets = [ tweet for index, tweet in enumerate(data['christchurchClean']) if data['christchurchSA'][index] < 0]
CCpos=format(len(CCpos_tweets)*100/len(data['christchurchClean']))
CCneu=format(len(CCneu_tweets)*100/len(data['christchurchClean']))
CCneg=format(len(CCneg_tweets)*100/len(data['christchurchClean']))

In [47]:
PLpos_tweets = [ tweet for index, tweet in enumerate(data_pl['parklandClean']) if data_pl['parklandSA'][index] > 0]
PLneu_tweets = [ tweet for index, tweet in enumerate(data_pl['parklandClean']) if data_pl['parklandSA'][index] == 0]
PLneg_tweets = [ tweet for index, tweet in enumerate(data_pl['parklandClean']) if data_pl['parklandSA'][index] < 0]
PLpos=format(len(PLpos_tweets)*100/len(data_pl['parklandClean']))
PLneu=format(len(PLneu_tweets)*100/len(data_pl['parklandClean']))
PLneg=format(len(PLneg_tweets)*100/len(data_pl['parklandClean']))

In [49]:
fig=plt.figure()
ax=fig.add_subplot(111,projection='3d')
plx,ply,plz= plot_sent(parkland)
ccx, ccy, ccz = plot_sent(christchurch)
ax.scatter(ccx, ccy, ccz,c='b', marker='o', label="Christchurch")
ax.scatter(plx,ply,plz,c='r', marker='x', label="Parkland")
ax.set_xlabel('Positive')
ax.set_ylabel('Neutral')
ax.set_zlabel('Negative')
leg = ax.legend();
fig.savefig("output.pdf", facecolor=fig.get_facecolor(), transparent=True)
plt.show()

<img src="sent-analysis.png">

The above 3d plot shows that most of the tweets were either positive or neutral for both the incidents whereas there were some that were negative. The negative tweets might be the one that share the experiences. 

## Most Frequently Used "Words"

In [31]:

PPlexemes = []
for tweet in data_pl['parklandClean']:
    PPlexemes.extend(tweet.split())
def lexical_diversity(tokens):
    return len(set(tokens))/len(tokens)
PPld=lexical_diversity(PPlexemes)
PPcommonwords=(Counter(PPlexemes).most_common(10))
df_parkland = pd.DataFrame.from_dict(PPcommonwords)
df_parkland=df_parkland.rename(columns={0:'ParklandCommon words', 1:'Parklandcount'})
df_parkland

,ParklandCommon words,Parklandcount
0,#MarchForOurLives,623
1,https,529
2,dnkGeM,188
3,#AARP,132
4,#veterans,132
5,#Millennials,117
6,#METOO,112
7,#Latinx,108
8,#LGBTQ,107
9,#TeaParty,101


In [20]:
CClexemes = []
for tweet in data['christchurchClean']:
    CClexemes.extend(tweet.split())
def lexical_diversity(tokens):
    return len(set(tokens))/len(tokens)
CCld=lexical_diversity(CClexemes)
CCcommonwords=(Counter(CClexemes).most_common(10))
df_christchurch = pd.DataFrame.from_dict(CCcommonwords)
df_christchurch=df_christchurch.rename(columns={0:'CCCommon words', 1:'CCcount'})
df_christchurch

,CCCommon words,CCcount
0,https,43
1,#NewZealand,41
2,#Christchurch,26
3,Zealand,26
4,#ChristchurchShooting,22
5,#ChristchurchMosqueAttack,22
6,#Christchurchshooting,21
7,#NewZealandShooting,20
8,#World,19
9,Mosque,18


Both the table shows 10 most frequently used words in the 10000 tweets gathered with the hashtag. This is a very interesting finding. Both the incidents have several other hastags attached to the hastag of the event. On one hand, Parkland shooting has touched various different areas which necessarily doesn't relate to the incident. For eg: #Metoo is a movement for sexual assault surviours or #Millenials. On the other hand, Christchurch shooting and teh hastags related are mainly focused on the same incident

## Text Anlaysis on the Speech given by the leader of respective countries

#### President Trump on Parkland Shooting

In [50]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
trumpSpeech=open ("trumpSpeech.txt").read()
wordcloud = WordCloud(collocations=False,max_words=50,stopwords=STOPWORDS, background_color='white',max_font_size=40, scale=3,color_func=random_color_func).generate(trumpSpeech)
plt.imshow(wordcloud,interpolation="bilinear") 
plt.axis('off')
plt.show()

<img src="trumpSpeech.png">
The above wordcloud shows randomly chosen 50 words from President Trump's speech after Parkland Shooting.

#### Prime Minister Arden on Christchurch Shooting

In [53]:
JacindaSpeech=open ("JacindaSpeech.txt").read()
wordcloud = WordCloud(collocations=False,max_words=50,stopwords=STOPWORDS, background_color='white',max_font_size=40, scale=3,color_func=random_color_func).generate(JacindaSpeech)
plt.imshow(wordcloud,interpolation="bilinear") 
plt.axis('off')
plt.show()

<img src="JacindaSpeech.png">
The above wordcloud shows randomly chosen 50 words from Prime Minister Ardern's speech after Christchurch Shooting.

## Conclusion

This project mainly compared the twitter reactions to such violent crimes. Even though these shooting happened in two differnt places on the world, Twitter did react similar way. The sentiment analysis of the tweets were very similar. BOth the leaders of the nations addresed the community and the families. Both the leaders talked about the nation. President Trump on one hand did not categorize this as an act of terrorism whereas Prime Minister Ardern did. The only major difference was the use of hashtag. Christchurch shooting hashtags were focused on the shooting itself whereas parkland shooting was focusing on other differnt areas which was not necessarily related to the shooting 